In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [104]:
import networkx as nx

In [204]:
import sqlite3
conn2 = sqlite3.connect("intermediate2.db")

In [48]:
# Elimiate relationships outside the list, and, 
# Elimiate relationships which the two parties' index year are bigger than 100 years.
def filting(relationlist, peoplelist, years):
    
    tempdf = relationlist.join(peoplelist, on='initiator')
    tempdf.columns = ['initiator', 'object', 'weight', 'iniYear', 'iniIn']
    tempdf = tempdf.join(peoplelist, on='object')
    tempdf.columns = ['initiator', 'object', 'weight','iniYear', 'iniIn', 'objYear', 'objIn' ]
    tempdf['keep'] = tempdf.iniIn | tempdf.objIn
    tempdf.drop(['iniIn','objIn'], axis = 1, inplace = True)
    edgelist = tempdf[tempdf.keep].drop('keep', axis = 1)
        
    return edgelist[abs(edgelist.iniYear - edgelist.objYear) < years].drop(['iniYear','objYear'],axis = 1)



In [3]:
databasename = "../Network Centrality/CBDB_aw_20180831_sqlite.db"
 
# create a database connection
conn = create_connection(databasename)
cur = conn.cursor()

In [52]:
peopleyear = pd.read_csv("../Network Centrality/LindexYear.csv").set_index('c_personid').dropna()
peopleyear.LindexYear = peopleyear.LindexYear.astype('int64')

# Get non-kin association 

In [157]:
# Use SQL to get association table
cur.execute("SELECT c_personid AS personA, c_assoc_id AS personB, c_assoc_code from assoc_data")

rows = cur.fetchall() 
# convert to panda dataframe
asso = pd.DataFrame(rows, columns=['initiator', 'object', 'category'])

# read weight definition file

asso_weight = pd.read_csv("../Network Centrality/assoc_codes_weight4.csv")

# drop all columns except association category and weight. 
asso_weight.drop(["c_assoc_pair", "c_assoc_desc", "c_assoc_desc_chn","c_assoc_role_type", "c_sortorder"], axis=1, inplace = True)

# Join weight into the association table
asso_weighted = asso.join(
    asso_weight.set_index("c_assoc_code"),
    on = "category"
)
# remove relationships with "nil" or "0" weight. 
asso_weighted = asso_weighted [asso_weighted["Weight"]!="Nil"]
asso_weighted = asso_weighted [asso_weighted["Weight"]!=0]

# dropassociation category
asso_weighted.drop("category", axis = 1, inplace = True)

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


# Kin table

In [64]:
# Use SQL to get association table
#cur.execute("SELECT c_personid AS personA, c_kin_id AS personB, c_kin_code from kin_data \
#    WHERE\
#    personA IN (\
#        SELECT c_personid FROM biog_main WHERE c_index_year <= 1912 AND c_index_year >= 618)\
#    AND\
#    c_kin_code IN (75, 82, 107, 111, 135, 138, 163, 168, 176, 180, 182, 183, 184, 185, 186, 188, 191, 193, 194, 195, 196, 198, 199, 202, 204, 205, 206, 207, 211, 212, 213, 220, 221, 222, 226, 227, 229, 230, 231, 234, 235, 307, 326, 327, 329, 334, 335, 339, 343, 344, 363, 373, 377, 378, 420, 436, 437, 439, 440, 450, 451, 452, 453, 454, 455, 456, 457, 458, 467, 468, 559, 560, 575);")

# Use SQL to get association table
cur.execute("SELECT c_personid AS personA, c_kin_id AS personB, c_kin_code from kin_data \
    WHERE\
    personA IN (\
        SELECT c_personid FROM biog_main WHERE c_index_year <= 3079 AND c_index_year >= 0)\
    AND\
    c_kin_code IN (75, 82, 107, 111, 135, 138, 163, 168, 176, 180, 182, 183, 184, 185, 186, 188, 191, 193, 194, 195, 196, 198, 199, 202, 204, 205, 206, 207, 211, 212, 213, 220, 221, 222, 226, 227, 229, 230, 231, 234, 235, 307, 326, 327, 329, 334, 335, 339, 343, 344, 363, 373, 377, 378, 420, 436, 437, 439, 440, 450, 451, 452, 453, 454, 455, 456, 457, 458, 467, 468, 559, 560, 575);")



rows = cur.fetchall() 

# convert to panda dataframe
dkin = pd.DataFrame(rows, columns=['initiator', 'object', 'category'])

# read weight definition file

dkin_weight = pd.read_csv("../Network Centrality/kincode_direct_w1.csv")

# drop all columns except association category and weight. 
dkin_weight.drop(["Name"], axis=1, inplace = True)

# Join weight into the association table
dkin_weighted = dkin.join(
    dkin_weight.set_index("kin_code"),
    on = "category"
)

# remove relationships with "nil" or "0" weight. 

dkin_weighted = dkin_weighted [dkin_weighted["Weight"]!=0]

# dropassociation category
dkin_weighted.drop("category", axis = 1, inplace = True)



# Process for each decade

In [94]:
def splitpeople(startyear, endyear, allpeople):
       

    chosenpeople = allpeople.copy()

    chosenpeople['final'] = (chosenpeople.LindexYear >startyear-100) & (chosenpeople.LindexYear <= endyear)

    # filter people near the people 
    filteredkin = filting(dkin_weighted.iloc[:,:3], chosenpeople, 100)
    filteredasso = filting(asso_weighted.iloc[:,:3], chosenpeople, 100)
    
    # concatenate two tables
    allasso = pd.concat([filteredkin,filteredasso], axis = 0)

    # create "weight" columns as float type based on "Weight" column and then drop "Weight" column
    allasso['weight'] = allasso.weight.astype('float')
    #allasso.drop('Weight', axis = 1, inplace = True)
    # add all weight from the same initiator to the same obeject together
    asso_grouped = allasso.groupby(["initiator","object"]).agg('sum').reset_index()
    
    return asso_grouped


# Calculate two scores

In [197]:
def calculateeigenvector(allrel):

    # split relationships to positive (assodf) and negative (enemydf)
    assodf = allrel[allrel.weight>0]
    enemydf = allrel[allrel.weight<0]
    
    # build a graph based on assodf
    G = nx.from_pandas_edgelist(assodf, 'initiator', 'object', ['weight'])
    # Calculate the eigenvector centrality (EVC) of assodf
    centrality = nx.eigenvector_centrality(G ,max_iter=500, weight = 'weight' )
   
    
    # Build a dataframe based on the eigenvector centrality 
    evc = pd.DataFrame(data = centrality,index=[0]).T
    

    evc.columns = ['originalEVC']
    # calculate the lowest EVC
    minEVC = min(evc.originalEVC)
    # set the lowest EVC to ONE and then Logarithm
    evc['transformedEV'] =  np.log(evc.originalEVC*np.e/minEVC)
    
    # save the eigenvector centrality to file
    #    evc.to_csv('eigenvectorPositive.csv',header = None)

    # find the EVC of the object for each relationship
    enemydf = enemydf.join(evc, on='object').fillna(1)
    # weighted the relationship with EVC of the object 
    enemydf['evcWeighted'] =  - enemydf.weight * enemydf.transformedEV
    enemydf.drop(['weight','originalEVC','transformedEV'], axis = 1, inplace = True)
    # add enemy index by initiator
    enemyIndex = enemydf.groupby(['initiator'])['evcWeighted'].agg('sum')
    enemyIndex.name = 'nemesis'
    #    enemyIndex.to_csv('enemyindex.csv')
    
    evc.drop(['originalEVC'], axis = 1, inplace = True)
    evc.columns = ['status']
    
    return evc, enemyIndex

# statistics

In [90]:
raw = pd.read_csv('Song1-9.5onehot.csv')
raw = raw[raw.Lindex != 0]

In [ ]:
dataset = raw.copy()

In [ ]:
for startingyear in range(960,1280,10):



In [209]:
for startingyear in range(1090,1091,10):
    peopleslink = splitpeople(startingyear,startingyear+10,peopleyear)
    peopleslink.to_sql('connection'+str(startingyear), con=conn2, if_exists='replace')    
    statues, nemesis = calculateeigenvector(peopleslink)
    dataset = raw[(raw.Lindex >startingyear) & (raw.Lindex <= startingyear+10)].copy()
    dataset = dataset.join(statues, on = 'c_personid').fillna(0)
    dataset = dataset.join(nemesis, on = 'c_personid').fillna(0)
    dataset.to_sql('dataset'+str(startingyear), con=conn2, if_exists='replace')

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 500 iterations')